In [71]:
'''
In this file, we calculate the von neumann entropy of a series of density matrices produced by quantum circuits stemming from diagrams that represent sentences. 

For this, we construct diagrams and circuits representing sentences. We then forget words in these sentences and plug in the parameters learned in the VQC training process 
for the remaining words in the sentence. 
From this, because there is a maximally mixed state involved (representing the forgotten word), we construct the density matrix using the Qiskit library. Then, we compute the
Von-Neumann-Entropy and the Fidelity (as explained in the paper) for the different density matrices. 
'''



'\nIn this file, we calculate the von neumann entropy of a series of diagrams. \n\nWe compare them to each other in the hope of finding that more ambiguous (less disambiguated sentences) have higher entropy. \n\n'

In [72]:
'''
start by importing all sorts of stuff
'''


import numpy as np
from sympy import Symbol, symbols

from lambeq.ansatz import BaseAnsatz
from lambeq.backend.grammar import Box, Diagram, Functor, Ty
from lambeq.backend.quantum import (
    Bra,
    CRz,
    Diagram as Circuit,
    Discard,
    H,
    CX,
    Id,
    Ket,
    quantum,
    qubit,
    Rotation,
    Rx, Ry, Rz
)

from lambeq import AtomicType


In [73]:
'''
Note that for this program, we do not have to import test data. 
We test the entropy for sentences crafted in the program especially for the program. 

We write three functions for parsing three types of sentences: 
1. Alice cooks food
2. Alice cooks food and Bob does too 
3. Alice cooks food and eats it. 
'''

from lambeq import BobcatParser
parser = BobcatParser(verbose='text')
from lambeq import RemoveCupsRewriter
remove_cups = RemoveCupsRewriter()
from lambeq import AtomicType, IQPAnsatz
ansatz = IQPAnsatz({AtomicType.NOUN: 1, AtomicType.SENTENCE: 1},
                   n_layers=1, n_single_qubit_params=3)

from lambeq.backend.grammar import (Box, Cap, Cup, Diagram, Diagrammable,
                                    Functor, grammar, Id, Spider, Swap,
                                    Ty, Word)
from lambeq import AtomicType
N = AtomicType.NOUN
S = AtomicType.SENTENCE
n, s = Ty('n'), Ty('s')

# create three functions for three types of diagrams

def GijsDiag(sentence:str): 
    '''
    This function, given a sentence of the form: 
    Subj Verb Obj and Subj* does too
    , returns a diagram that copies the information in such a way that the returned diagram reads: 
    Subj Verb Obj and Subj* Verb Obj
    The object will be removed later to work with density matrices. 
    '''
    wordList = sentence.split()
    grammar = (Spider(n, 2, 1) @ Id(n.r) @ Id(s) @ Cup(n.l, n) >> 
            Cup(n, n.r) @ Id(s) )
    newWords = [
        Word(wordList[0], n),
        # Word('test', n),
        Word(wordList[4], n),
        Word(wordList[1], n.r @ s @ n.l), 
        Word(wordList[2], n)
    ]

    diagram = Id().tensor(*newWords) >> grammar
    diagram = remove_cups(diagram)
    # diagram.draw()

    circuit = ansatz(diagram= diagram)

    return circuit#, diagram 

def NaiveDiag(sentence:str):
    '''
    Simply returns a naive circuit given a diagram
    '''
    diagram = parser.sentence2diagram(sentence= sentence, suppress_exceptions=True)

    diagram = remove_cups(diagram)

    # diagram.draw()

    circuit = ansatz(diagram= diagram)

    return circuit#, diagram

def AmbiguousVerb(sentence:str):
    '''
    This function, given a sentence like: 
    Subj Verb Obj and Verb* it. (Man prepares dinner and eats it.) 
    returns a diagram that copies the information, so that: 
    Subj Verb Obj and Subj Verb* Obj
    The object will be removed later 
    '''
    wordList = sentence.split()
    grammar = (Spider(n, 1, 2) @ Id(n.r) @ Id(s) @ Id(n.l) @ Spider(n, 1, 2)  >> 
                Id(n) @ Cup(n, n.r) @ Id(s) @ Cup(n.l, n) @ Id(n) >> 
                Id(n) @ Word(wordList[4], n.r @ s @ n.l) @ Id(s) @ Id(n) >> 
                Cup(n, n.r) @ Id(s) @ Swap(n.l, s) @ Id(n) >> 
                Spider(s, 2, 1) @ Cup(n.l, n))
    newWords = [
        Word(wordList[0], n),
        Word(wordList[1], n.r @ s @ n.l),
        Word(wordList[2], n)
    ]


    diagram = Id().tensor(*newWords) >> grammar
    diagram = remove_cups(diagram)
    # diagram.draw()
    circuit = ansatz(diagram= diagram)

    return circuit#, diagram


    




In [74]:
def AmbiguousVerb2(sentence:str): 
    '''
    
    '''
    wordList = sentence.split()
    newWords = [
        # Word(wordList[0], n),
        Word(wordList[1], n.r @ s @ n.l),
        Word(wordList[4], n.r @ s @ n.l),
        # Word(wordList[2], n)
    ]

    grammar = (
        Id(n.r) @ Id(s) @ Swap(n.l, n.r) @ Id(s) @ Id(n.l) >> 
        Id(n.r) @ Swap(s, n.r) @ Swap(n.l, s) @ Id(n.l) >> 
        Spider(n.r, 2, 1) @ Spider(s, 2, 1) @ Spider(n.l, 2, 1) >> 
        Word(wordList[0], n.r).dagger() @ Id(s) @ Word(wordList[2], n.l).dagger()
    )


    # grammar = (
    #     Spider(n, 1, 2) @ Id(n.r) @ Id(s) @ Id(n.l) @ Id(n.r) @ Id(s) @ Id(n.l) @ Spider(n, 1, 2) >>
    #     Id(n) @ Cup(n, n.r) @ Id(s) @ Id(n.l) @ Id(n.r) @ Id(s) @ Cup(n.l, n) @ Id(n) 
    # )

    diagram = Id().tensor(*newWords) >> grammar
    # diagram = remove_cups(diagram)
    diagram.draw()
    circuit = ansatz(diagram= diagram)

    return circuit#, diagram

In [75]:

GIJS = True  

# naiveSentences = ['person prepares meal', 'person cooks meal', 'person bakes meal', 'person reads meal', 'person runs meal'
#                 'person prepares dinner', 'person cooks dinner', 'person bakes dinner', 'person reads dinner', 'person runs dinner',
#                 'person prepares sauce', 'person cooks sauce', 'person bakes sauce', 'person reads sauce', 'person runs sauce',
#                 'person prepares program', 'person cooks program', 'person bakes program', 'person reads program', 'person runs program',
#                 'person prepares software', 'person cooks software', 'person bakes software', 'person reads software', 'person runs software',
#                 'person prepares application', 'person cooks application', 'person bakes application', 'person reads application', 'person runs application',
#                 ]





if not GIJS: 

    senseFulSentences = ['person prepares meal', 'person cooks meal', 'person bakes meal',
                    'person prepares dinner', 'person cooks dinner', 'person bakes dinner',
                    'person prepares sauce', 'person cooks sauce', 'person bakes sauce',
                    'person prepares program', 'person reads program', 'person runs program',
                    'person prepares software', 'person reads software', 'person runs software',
                    'person prepares application', 'person reads application', 'person runs application',
                    'man prepares meal', 'man cooks meal', 'man bakes meal',
                    'man prepares dinner', 'man cooks dinner', 'man bakes dinner',
                    'man prepares sauce', 'man cooks sauce', 'man bakes sauce',
                    'man prepares program', 'man reads program', 'man runs program',
                    'man prepares software', 'man reads software', 'man runs software',
                    'man prepares application', 'man reads application', 'man runs application',
                    'woman prepares meal', 'woman cooks meal', 'woman bakes meal',
                    'woman prepares dinner', 'woman cooks dinner', 'woman bakes dinner',
                    'woman prepares sauce', 'woman cooks sauce', 'woman bakes sauce',
                    'woman prepares program', 'woman reads program', 'woman runs program',
                    'woman prepares software', 'woman reads software', 'woman runs software',
                    'woman prepares application', 'woman reads application', 'woman runs application',
                    ]
    
    # mySentences = ['person prepares meal and cooks it', 'person prepares meal and bakes it', 'person prepares meal and reads it', 'person prepares meal and runs it',
    #                 'man prepares meal and cooks it', 'man prepares meal and bakes it', 'man prepares meal and reads it', 'man prepares meal and runs it',
    #                 'woman prepares meal and cooks it', 'woman prepares meal and bakes it', 'woman prepares meal and reads it', 'woman prepares meal and runs it',
    #                 'person prepares meal and cooks it', 'person prepares meal and bakes it', 'person prepares meal and reads it', 'person prepares meal and runs it',
    #                 'man prepares meal and cooks it', 'man prepares meal and bakes it', 'man prepares meal and reads it', 'man prepares meal and runs it',
    #                 'woman prepares meal and cooks it', 'woman prepares meal and bakes it', 'woman prepares meal and reads it', 'woman prepares meal and runs it']


    mySentences = ['person prepares meal and cooks it', 'person prepares meal and bakes it',
                    'person prepares dinner and cooks it', 'person prepares dinner and bakes it',
                    'person prepares sauce and cooks it', 'person prepares sauce and bakes it',
                    'person prepares program and runs it', 'person prepares program and reads it',
                    'person prepares software and runs it', 'person prepares software and reads it',
                    'person prepares application and runs it', 'person prepares application and reads it',
                    'woman prepares meal and cooks it', 'woman prepares meal and bakes it',
                    'woman prepares dinner and cooks it', 'woman prepares dinner and bakes it',
                    'woman prepares sauce and cooks it', 'woman prepares sauce and bakes it',
                    'woman prepares program and runs it', 'woman prepares program and reads it',
                    'woman prepares software and runs it', 'woman prepares software and reads it',
                    'woman prepares application and runs it', 'woman prepares application and reads it',
                    'man prepares meal and cooks it', 'man prepares meal and bakes it',
                    'man prepares dinner and cooks it', 'man prepares dinner and bakes it',
                    'man prepares sauce and cooks it', 'man prepares sauce and bakes it',
                    'man prepares program and runs it', 'man prepares program and reads it',
                    'man prepares software and runs it', 'man prepares software and reads it',
                    'man prepares application and runs it', 'man prepares application and reads it',
                    ]

# gijs doesn't know words man, woman 
if GIJS:
    senseFulSentences = ['person prepares meal', 'person cooks meal', 'person bakes meal',
                    'person prepares dinner', 'person cooks dinner', 'person bakes dinner',
                    'person prepares sauce', 'person cooks sauce', 'person bakes sauce',
                    'person prepares program', 'person reads program', 'person runs program',
                    'person prepares software', 'person reads software', 'person runs software',
                    'person prepares application', 'person reads application', 'person runs application',
                    'chef prepares meal', 'chef cooks meal', 'chef bakes meal',
                    'chef prepares dinner', 'chef cooks dinner', 'chef bakes dinner',
                    'chef prepares sauce', 'chef cooks sauce', 'chef bakes sauce',
                    'programmer prepares program', 'programmer reads program', 'programmer runs program',
                    'programmer prepares software', 'programmer reads software', 'programmer runs software',
                    'programmer prepares application', 'programmer reads application', 'programmer runs application'
                    ]



    mySentences = ['person prepares meal and cooks it', 'person prepares meal and bakes it', 'person prepares meal and reads it', 'person prepares meal and runs it', 
                   'person prepares meal and cooks it', 'person prepares meal and bakes it', 'person prepares meal and reads it', 'person prepares meal and runs it']

    mySentences = ['person prepares meal and cooks it', 'person prepares meal and bakes it',
                    'person prepares dinner and cooks it', 'person prepares dinner and bakes it',
                    'person prepares sauce and cooks it', 'person prepares sauce and bakes it',
                    'person prepares program and runs it', 'person prepares program and reads it',
                    'person prepares software and runs it', 'person prepares software and reads it',
                    'person prepares application and runs it', 'person prepares application and reads it']



# we disambiguate the first sentence using the second one 
gijsSentences = ['person prepares meal and chef does too', 'person prepares dinner and chef does too', 'person prepares sauce and chef does too',
                 'person prepares application and programmer does too', 'person prepares software and programmer does too', 'person prepares program and programmer does too']
                #  ]#, 'person prepares meal and person does too']



naiveCircuits = []
gijsCircuits  = []
myCircuits    = []


for nS in senseFulSentences: 
    naiveCircuits.append(NaiveDiag(nS))

for gS in gijsSentences: 
    gijsCircuits.append(GijsDiag(gS))

for mS in mySentences: 
    myCircuits.append(AmbiguousVerb(mS))


  

In [76]:
# for circuit in naiveCircuits: 
#     circuit.draw()
# gijsDiags[0].draw()
# gijsCircuits[0].draw()
# myCircuits[0].draw()
# naiveCircuits[0].draw()

In [77]:
'''
import the functions needed to set up the diagrams using the trained weights 
'''

from entropyHelpers import *


In [78]:
'''
import trained weights 
'''
# one of these two is wordsToForget
wordsToForgetSubj = ['man', 'woman', 'person', 'chef', 'programmer']
wordsToForgetObj  = ['dinner', 'meal', 'sauce', 'program', 'application', 'software']


# don't use these words together with chef or something. 
# do not use words in the same list taht appear in the same sentence(s) together

# load dictionary with pickle
import pickle 

# also AE is an option: 
AE = True

# GIJS = False

if not AE: 
    if GIJS: 
        # Numpymodel 
        with open('../weights/weightsDictNP1QB_gijs_500.pkl', 'rb') as fp: 
            parameterDictNP = pickle.load(fp)

        # Numpymodel 
        with open('../weights/weightsDictTK1QB_gijs_500.pkl', 'rb') as fp: 
            parameterDictTK = pickle.load(fp)

        # Pennylane model 
        with open('../weights/weightsDictPL1QB_gijs_500.pkl', 'rb') as fp: 
            parameterDictPL = pickle.load(fp)

    if not GIJS: 
        # Numpymodel 
        with open('../weights/weightsDictNP1QB_500.pkl', 'rb') as fp: 
            parameterDictNP = pickle.load(fp)

        # TKet model 
        with open('../weights/weightsDictTK1QB_500.pkl', 'rb') as fp: 
            parameterDictTK = pickle.load(fp)

        # Pennylane model 
        with open('../weights/weightsDictPL1QB_500.pkl', 'rb') as fp: 
            parameterDictPL = pickle.load(fp)

 

if AE: 
    # Numpymodel 
    with open('../weights/weightsDict_NP_1QB_AE_500.pkl', 'rb') as fp: 
        parameterDictNP = pickle.load(fp) 

    # TKet model 
    with open('../weights/weightsDict_TK_1QB_AE_500_gijs.pkl', 'rb') as fp: 
        parameterDictTK = pickle.load(fp)

    # Pennylane model 
    with open('../weights/weightsDict_PL_1QB_AE_500_gijs.pkl', 'rb') as fp: 
        parameterDictPL = pickle.load(fp)


print(parameterDictNP)


{'bakes__n.r@s@n.l_0': 0.3938767877236, 'bakes__n.r@s@n.l_1': 0.910087234625596, 'cooks__n.r@s@n.l_0': 1.0011438686472736, 'cooks__n.r@s@n.l_1': 0.1512780168410606, 'prepares__n.r@s@n.l_0': 0.7777270085694745, 'prepares__n.r@s@n.l_1': 0.39811247906192704, 'reads__n.r@s@n.l_0': 0.8894986056496768, 'reads__n.r@s@n.l_1': 0.7458064421873172, 'runs__n.r@s@n.l_0': 0.9121203547968613, 'runs__n.r@s@n.l_1': 0.8025270232801771, 'skillful†_n@n.l__0': 0.7897751721225819, 'tasty†_n@n.l__0': 0.12279668996178265, 'useful†_n@n.l__0': 0.5151309729356118}


In [79]:
'''
apply the functions
'''

# parameterDict = parameterDictNP
parameterDict = parameterDictTK
# parameterDict = parameterDictPL

print(parameterDict)

naiveDMListForgetSubj = []
naiveDMListForgetObj  = []
gijsDMListForgetObj = []
gijsDMListForgetSubj = []
myDMListForgetSubj = []
myDMListForgetObj  = [] 

myFDListForgetSubj1QB = []
myFDListForgetObj1QB  = [] 

naiveFDListForgetSubj1QB = []
naiveFDListForgetObj1QB  = [] 
gijsFDListForgetSubj1QB = []
gijsFDListForgetObj1QB  = [] 


for nC in naiveCircuits: 

    subjResult = Main(listOfCircuits= [nC], parameterDict= parameterDict, wordsToForget=wordsToForgetSubj, amplitudeEncoded = AE)
    objResult = Main(listOfCircuits= [nC], parameterDict= parameterDict, wordsToForget=wordsToForgetObj, amplitudeEncoded = AE)

    naiveDMListForgetSubj.append(subjResult[0])
    naiveDMListForgetObj.append(objResult[0])

    naiveFDListForgetObj1QB.append(objResult[2])
    naiveFDListForgetSubj1QB.append(subjResult[2])


# commented this out

if GIJS:
    print('GIJS')
    for gC in gijsCircuits: 

        subjResult = Main(listOfCircuits= [gC], parameterDict= parameterDict, wordsToForget= ['person'], amplitudeEncoded = AE)
        objResult = Main(listOfCircuits= [gC], parameterDict= parameterDict, wordsToForget=wordsToForgetObj, amplitudeEncoded = AE)


        gijsDMListForgetObj.append(objResult[0])
        gijsDMListForgetSubj.append(subjResult[0])

        gijsFDListForgetObj1QB.append(objResult[2])
        gijsFDListForgetSubj1QB.append(subjResult[2])

print('MY')

for mC in myCircuits:

    subjResult = Main(listOfCircuits= [mC], parameterDict= parameterDict, wordsToForget=wordsToForgetSubj, amplitudeEncoded = AE)

    myDMListForgetSubj.append(subjResult[0])
    myFDListForgetSubj1QB.append(subjResult[2])


    objResult = Main(listOfCircuits= [mC], parameterDict= parameterDict, wordsToForget=wordsToForgetObj, amplitudeEncoded = AE)

    myDMListForgetObj.append(objResult[0])
    myFDListForgetObj1QB.append(objResult[2])
    # myDMList.append(Main(listOfCircuits= [mC], parameterDict= parameterDict, wordsToForget=wordsToForgetSubj)[0])


{'bakes__n.r@s@n.l_0': 0.33551560476558906, 'bakes__n.r@s@n.l_1': 0.7172257838442746, 'cooks__n.r@s@n.l_0': 0.4306531189949587, 'cooks__n.r@s@n.l_1': 0.8466301455401205, 'prepares__n.r@s@n.l_0': 0.8606360385553892, 'prepares__n.r@s@n.l_1': 0.595505338999644, 'reads__n.r@s@n.l_0': 0.830502516928964, 'reads__n.r@s@n.l_1': 0.8482639099644077, 'runs__n.r@s@n.l_0': 1.0478535349709972, 'runs__n.r@s@n.l_1': 0.8975000577889296, 'skillful†_n@n.l__0': 0.9410437778073346, 'tasty†_n@n.l__0': 0.6813356414275431, 'useful†_n@n.l__0': 0.5150674745512449}
iteration number:  0
circuit that we start with: 
     ┌───┐                                               ┌───┐              »
q_0: ┤ H ├────────────────■──────────────────────────────┤ H ├──────────────»
     ├───┤┌───────────────┴───────────────┐              └───┘              »
q_1: ┤ H ├┤ Rz(2*π*prepares__n.r@s@n.l_0) ├────────────────■────────────────»
     ├───┤└───────────────────────────────┘┌───────────────┴───────────────┐»
q_2: ┤ H ├─────

/home/bob/anaconda3/envs/code/lib/python3.9/site-packages/qiskit/visualization/circuit/matplotlib.py:266: FutureWarning: The default matplotlib drawer scheme will be changed to "iqp" in a following release. To silence this warning, specify the current default explicitly as style="clifford", or the new default as style="iqp".
  self._style, def_font_ratio = load_style(self._style)


removed measurement. 
trace out instead of measure, circuit is: 
     ┌───┐                  ┌───┐                                     
q_0: ┤ H ├──────■───────────┤ H ├─────────────────────────────────────
     ├───┤┌─────┴──────┐    └───┘     ┌───┐                           
q_1: ┤ H ├┤ Rz(5.4075) ├──────■───────┤ H ├───────────────────────────
     ├───┤└────────────┘┌─────┴──────┐├───┤┌───────┐┌───────┐┌───────┐
q_2: ┤ H ├──────────────┤ Rz(3.7417) ├┤ H ├┤ Rx(0) ├┤ Rz(0) ├┤ Rx(0) ├
     └───┘              └────────────┘└───┘└───────┘└───────┘└───────┘
c: 2/═════════════════════════════════════════════════════════════════
                                                                      
And traceList is:  [2]
debug, shape density matrix:  DensityMatrix([[ 1.91325868e-02+3.46944695e-18j,
                -1.32816771e-02-4.83668583e-02j,
                 1.07605479e-02-2.95487712e-03j,
                 0.00000000e+00-8.59977689e-02j,
                 4.77549970e-03-1.35303998e-02j

In [80]:
from qiskit.quantum_info import entropy, purity
print(myDMListForgetObj)
naiveEntropyListForgetSubj = []
naiveEntropyListForgetObj  = []
gijsEntropyListForgetObj  = []
gijsEntropyListForgetSubj = []
myEntropyListForgetSubj    = []
myEntropyListForgetObj     = []

for dM1 in naiveDMListForgetSubj: 
    # print(dM1)
    naiveEntropyListForgetSubj.append(entropy(dM1[0]))

for dM2 in naiveDMListForgetObj: 
    naiveEntropyListForgetObj.append(entropy(dM2[0]))

for dM in gijsDMListForgetObj: 
    gijsEntropyListForgetObj.append(entropy(dM[0], base = np.e))

for dM in gijsDMListForgetSubj: 
    gijsEntropyListForgetSubj.append(entropy(dM[0]))

for dM3 in myDMListForgetSubj: 
    myEntropyListForgetSubj.append(entropy(dM3[0]))

for dM4 in myDMListForgetObj: 
    myEntropyListForgetObj.append(entropy(dM4[0]))


# print(gijsDMListForgetObj[0])
# print(gijsEntropyListForgetObj)


[[DensityMatrix([[ 0.6679653 -4.32442531e-17j, -0.13059191-3.87803731e-01j],
               [-0.13059191+3.87803731e-01j,  0.3320347 -2.87219293e-17j]],
              dims=(2,))], [DensityMatrix([[0.50277021-1.23853108e-17j, 0.24709841-3.28347696e-01j],
               [0.24709841+3.28347696e-01j, 0.49722979+1.45709539e-17j]],
              dims=(2,))], [DensityMatrix([[ 0.40482721+9.34523392e-18j, -0.08849926+4.20258259e-01j],
               [-0.08849926-4.20258259e-01j,  0.59517279-2.20887347e-17j]],
              dims=(2,))], [DensityMatrix([[0.19882228-4.35744847e-17j, 0.29425204-1.58587363e-01j],
               [0.29425204+1.58587363e-01j, 0.80117772-1.83747827e-18j]],
              dims=(2,))], [DensityMatrix([[ 0.66973085-7.23126625e-17j, -0.02812564-4.10766594e-01j],
               [-0.02812564+4.10766594e-01j,  0.33026915-2.27500511e-17j]],
              dims=(2,))], [DensityMatrix([[0.53830585-3.50705085e-17j, 0.22759798-3.34118129e-01j],
               [0.22759798+3.34118129e

In [81]:

# collect both entropy and fidelities 


# we now calculalte the fidelity of the dms with the respective pure density matrices corresponding to the both categories 

from qiskit.quantum_info import state_fidelity
from qiskit.quantum_info import partial_trace

iTMatrix = np.array([[1,0], [0,0]])
foodMatrix = np.array([[0,0], [0,1]])
iTDM = DensityMatrix(iTMatrix)
foodDM = DensityMatrix(foodMatrix)


naiveFidelitiesSubj = []
naiveFidelitiesObj = []

myFidelitiesSubj = []
myFidelitiesObj = []

gijsFidelitiesSubj = []
gijsFidelitiesObj = [] 

# 
# NAIVE!! 
# 

objList = []
subjList = []
for i in range(len(senseFulSentences)):

    
    if 'prepares' in senseFulSentences[i]:
        objList.append(naiveEntropyListForgetObj[i])
        subjList.append(naiveEntropyListForgetSubj[i])

        

        # # if the sentence is in the category food
        if 'meal' in senseFulSentences[i] or 'sauce' in senseFulSentences[i] or 'dinner' in senseFulSentences[i] or 'chef' in senseFulSentences[i] or 'bakes' in senseFulSentences[i] or 'cooks' in senseFulSentences[i]:
            naiveFidelitiesObj.append(naiveFDListForgetObj1QB[i][1])
            naiveFidelitiesSubj.append(naiveFDListForgetSubj1QB[i][1])
        else: 
            naiveFidelitiesObj.append(naiveFDListForgetObj1QB[i][0])
            naiveFidelitiesSubj.append(naiveFDListForgetSubj1QB[i][0])



test1 = []
test2 = []
# my sentences fidelities 
for i in range(len(myDMListForgetObj)):  

    if 'cooks' in mySentences[i] or 'bakes' in mySentences[i]: 
        print('one')
        myFidelitiesObj.append(myFDListForgetObj1QB[i][1])
        myFidelitiesSubj.append(myFDListForgetSubj1QB[i][1])
        test1.append(myFDListForgetObj1QB[i][1])
        test2.append(myFDListForgetSubj1QB[i][1])

    else: 
        print('two')
        test1.append(myFDListForgetObj1QB[i][0])
        test2.append(myFDListForgetSubj1QB[i][0])
        myFidelitiesObj.append(myFDListForgetObj1QB[i][0])
        myFidelitiesSubj.append(myFDListForgetSubj1QB[i][0])

print('sums')
print(sum(test1) / len(test1))
print(sum(test2) / len(test2))


# # gijs sentences fidelity 
for i in range(len(gijsDMListForgetObj)):  

#     objDM = gijsDMListForgetObj[i][0]
#     objDM = partial_trace(objDM, [1])

#     subjDM = gijsDMListForgetSubj[i][0]
#     subjDM = partial_trace(subjDM, [0])

    if 'chef' in gijsSentences[i]: 
        gijsFidelitiesObj.append(gijsFDListForgetObj1QB[i][1])
        gijsFidelitiesSubj.append(gijsFDListForgetSubj1QB[i][1])
    else: 
        gijsFidelitiesObj.append(gijsFDListForgetObj1QB[i][0])
        gijsFidelitiesSubj.append(gijsFDListForgetSubj1QB[i][0])



# 
# APPEND THE FIDELITIES AS DOWNSTAIRS!! 
# 

print('avg fidelities naive: ')
print('obj')
print(sum( naiveFidelitiesObj) / len(naiveFidelitiesObj))
print('subj')
print(sum( naiveFidelitiesSubj) / len(naiveFidelitiesSubj))


print('avg fidelities my sentences: ')
print('obj')
print(sum( myFidelitiesObj) / len(myFidelitiesObj))
print('subj')
print(sum( myFidelitiesSubj) / len(myFidelitiesSubj))

if GIJS: 
    print('avg fidelities gijs: ')
    print('obj')
    print(sum( gijsFidelitiesObj) / len(gijsFidelitiesObj))
    print('subj')
    print(sum( gijsFidelitiesSubj) / len(gijsFidelitiesSubj))


    
print('### naive sentencnes ###')
print('forget object: ')
print(sum(objList)/len(objList))
print('forget subject: ')
print(sum(subjList)/ len(subjList))
# print(myEntropyListForgetObj)

print('### my sentences ###')
print('forget object: ', sum(myEntropyListForgetObj)/len(myEntropyListForgetObj))
print('forget subject: ', sum(myEntropyListForgetSubj)/len(myEntropyListForgetSubj))


if GIJS:
    print('### gijs DMs ###')
    print('forget object: ') 
    print(sum(gijsEntropyListForgetObj)/ len(gijsEntropyListForgetObj))
    print('forget subject: ')
    print(sum(gijsEntropyListForgetSubj)/ len(gijsEntropyListForgetSubj))


# print fidelities! 

print(gijsDMListForgetObj)
print(gijsDMListForgetSubj)
print(gijsFDListForgetObj1QB)
print(gijsFDListForgetSubj1QB)


one
one
one
one
one
one
two
two
two
two
two
two
sums
0.727018177678595
0.6633364446343458
avg fidelities naive: 
obj
0.5178206125038002
subj
0.5585931821335081
avg fidelities my sentences: 
obj
0.727018177678595
subj
0.6633364446343458
avg fidelities gijs: 
obj
0.5014249006245751
subj
0.5613417890451409
### naive sentencnes ###
forget object: 
0.8900319304897141
forget subject: 
0.09862281618813601
### my sentences ###
forget object:  0.2658558357544549
forget subject:  0.16105976678649292
### gijs DMs ###
forget object: 
0.6488108947894621
forget subject: 
0.08790864037748718
[[DensityMatrix([[3.66160401e-01+0.j        , 3.06192242e-17+0.06265379j],
               [6.12384483e-17-0.06265379j, 6.33839599e-01+0.j        ]],
              dims=(2,))], [DensityMatrix([[3.66160401e-01+0.j        , 3.06192242e-17+0.06265379j],
               [6.12384483e-17-0.06265379j, 6.33839599e-01+0.j        ]],
              dims=(2,))], [DensityMatrix([[3.66160401e-01+0.j        , 3.06192242e-17+0.062